In [1]:
# importing all necessary libs
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objs as go
import dash
import os
import dash_core_components as dcc
import dash_html_components as html
from plotly.subplots import make_subplots
import py7zr
import warnings
warnings.filterwarnings("ignore")


In [2]:
curr_dir = os.getcwd()
parent_dir = os.path.dirname(curr_dir)

# Data reading
with py7zr.SevenZipFile(parent_dir + '/data/globalterrorismdb_0221dist.7z') as z:
    # open the csv file in the dataset
    targetPath = os.path.expanduser("~/Desktop/GT Dataset")
    z.extract(path = targetPath)
    df = pd.read_excel(targetPath + '/globalterrorismdb_0221dist.xlsx')
    
# We filtered out doubted attacks to be able to have exact attacks
df = df[df['doubtterr'] != 1]

_df = df[['eventid','iyear','imonth','iday','country','country_txt',
         'region','region_txt', 'provstate', 'city','latitude','longitude',
         'gname','nkill', 'nwound']]

In [3]:
# Geojson for polygons of map
with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
    countries = json.load(response)
                          
countries_df = pd.json_normalize(countries,  record_path =['features'])

In [4]:
# Updating the countries to be able to merge -- TODO: !!!!
_df.loc[_df['country_txt'] == 'Bahamas','country_txt']  = 'The Bahamas'
_df.loc[_df['country_txt'] == 'United States','country_txt'] = 'United States of America'
_df.loc[_df['country_txt'] == 'Czechoslovakia','country_txt'] = 'Czech Republic'
_df.loc[_df['country_txt'] == 'East Germany (GDR)','country_txt']  = 'Germany'
_df.loc[_df['country_txt'] == 'West Germany (FRG)','country_txt']  = 'Germany'

_df.loc[_df['country_txt'] == 'West Bank and Gaza Strip','country_txt'] = 'West Bank'
_df.loc[_df['country_txt'] == 'South Vietnam','country_txt'] = 'Vietnam'
_df.loc[_df['country_txt'] == 'North Yemen','country_txt'] = 'Yemen'
_df.loc[_df['country_txt'] == 'South Yemen','country_txt'] = 'Yemen'
_df.loc[_df['country_txt'] == 'Andorra','country_txt'] = 'Spain'
_df.loc[_df['country_txt'] == 'Bahrain','country_txt'] = 'Iran'
_df.loc[_df['country_txt'] == 'Tanzania','country_txt'] = 'United Republic of Tanzania'

for ind in _df.index:
    if _df.loc[ind,'country_txt'] in ['Soviet Union', 'Yugoslavia']:
        _df.loc[ind,'country_txt'] = _df.loc[ind,'provstate']


_df.loc[_df['country_txt'] == 'Bosnia-Herzegovina','country_txt'] = 'Bosnia and Herzegovina'
_df.loc[_df['country_txt'].isin(['Central Serbia','Serbia','Serbia-Montenegro',
                                 'Republika Srpska','Belgrade','Kosovo and Metohija']),'country_txt'] = 'Republic of Serbia'  
_df.loc[_df['country_txt'] == 'Yugoslavia','country_txt'] = 'Republic of Serbia'
_df.loc[_df['country_txt'] == 'International','country_txt'] = 'United Arab Emirates'
_df.loc[_df['country_txt'] == 'Hong Kong','country_txt'] = 'China' 
_df.loc[_df['country_txt'] == 'Kygyzstan','country_txt'] = 'Kyrgyzstan'
_df.loc[_df['country_txt'] == 'Maldives','country_txt'] = 'India'

In [5]:
# Seychelles , --> no wound and kill
_df[_df['country_txt']=='Maldives']

,eventid,iyear,imonth,iday,country,country_txt,region,region_txt,provstate,city,latitude,longitude,gname,nkill,nwound


In [6]:
countries_df['properties.name'].unique()

array(['Afghanistan', 'Angola', 'Albania', 'United Arab Emirates',
       'Argentina', 'Armenia', 'Antarctica',
       'French Southern and Antarctic Lands', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'The Bahamas', 'Bosnia and Herzegovina',
       'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Brunei',
       'Bhutan', 'Botswana', 'Central African Republic', 'Canada',
       'Switzerland', 'Chile', 'China', 'Ivory Coast', 'Cameroon',
       'Democratic Republic of the Congo', 'Republic of the Congo',
       'Colombia', 'Costa Rica', 'Cuba', 'Northern Cyprus', 'Cyprus',
       'Czech Republic', 'Germany', 'Djibouti', 'Denmark',
       'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt', 'Eritrea',
       'Spain', 'Estonia', 'Ethiopia', 'Finland', 'Fiji',
       'Falkland Islands', 'France', 'Gabon', 'United Kingdom', 'Georgia',
       'Ghana', 'Guinea', 'Gambia', 'Guinea Bissau', 'Equatorial Guinea

In [7]:
set(_df['country_txt'])-set(countries_df['properties.name'])

{'Antigua and Barbuda',
 'Barbados',
 'Branicevo',
 'Bujanovac',
 'Comoros',
 'Crna Gora',
 'Dagestan',
 'Dominica',
 'French Polynesia',
 'Grenada',
 'Guadeloupe',
 'Guinea-Bissau',
 'Jablanica',
 'Kolubara',
 'Macau',
 'Martinique',
 'Mauritius',
 'New Hebrides',
 'Pcinja',
 "People's Republic of the Congo",
 'Podgorica',
 'Presevo',
 'Rhodesia',
 'Seychelles',
 'Singapore',
 'Slovak Republic',
 'St. Kitts and Nevis',
 'St. Lucia',
 'Unknown',
 'Vatican City',
 'Vojvodina',
 'Vranje',
 'Wallis and Futuna',
 'Zaire'}

In [9]:
# Merging locations and main df
df_merged = pd.merge(_df, countries_df , left_on = 'country_txt', right_on = 'properties.name', how = 'left' )

# Determine ig organized or unorganized
df_merged['isOrganized'] = ['unorganized' if i == 'Unknown' else 'organized' for i in df_merged['gname']]

weights = np.arange(1, len(df_merged.iyear.unique())+1, step = 1)
years = df_merged.iyear.unique()
weighted_years = pd.DataFrame({'years': years, 'weights': weights})

df_merged = pd.merge(df_merged, weighted_years, left_on = 'iyear', right_on = 'years', how = 'inner')
df_merged['calculated_index'] =(( df_merged['nkill']*  df_merged['weights'] + df_merged['nwound']*  df_merged['weights'] )/df_merged['weights'])


In [10]:
# We need safety index calculation before groupby
df_map = df_merged.groupby(by=['country','country_txt','id']).agg({'calculated_index':np.mean}).reset_index()

def update_map():
    df_map['hover_text'] = 'Country: ' + df_map["country_txt"].astype(str) + "\n" + 'Safety Index: ' + df_map['calculated_index'].round(decimals= 2).astype(str)
    
    trace = go.Choropleth(locations=df_map['id'],
                          z=df_map['calculated_index'],
                          text=df_map['hover_text'], 
                          hoverinfo='text' ,
                          marker_line_color='white',
                          colorscale="Reds",
                          reversescale = False,
                          marker={'line': {'color': 'rgb(180,180,180)','width': 0.5}},
                          colorbar={"thickness": 20,"len": 0.7, "x": 0.9, "y": 0.7,
                                   'title': {"text": 'Safety Index', "side": "bottom"},
                                 }
                         )   
    return {"data": [trace],
            "layout": go.Layout(height=800, width= 1200 ,
                                geo={'showframe': False,
                                     'showcoastlines': False,
                                     'projection': {'type': "miller"}})}



external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Dashboard'


app.layout = html.Div([
    html.Div([html.H1("Global Terrorism Dashboard")],
                 style={'textAlign': "center", "padding-bottom": "30"}, className="six-column"),
    html.Div(dcc.Graph(id="world-map", style = {'textAlign': "center"}, figure = update_map()), 
             className="twelve columns"),
    html.Div(dcc.Graph(id="horizontal-bar"), className="twelve columns"),
    html.H2(id = 'country-name',  children=["init"] , style={'textAlign': "center", "padding-bottom": "30"} ),
    html.Div(dcc.Graph(id="incidents"),  id = 'incident-box', style = {'visibility': 'hidden'}, className="five columns"),
    html.Div(dcc.Graph(id="fatalities"),  id = 'fatalities-box', style = {'visibility': 'hidden'}, className="five columns"),
    
])


##### Figure Updates with CallBacks ##### 
@app.callback(
    dash.dependencies.Output('country-name', 'children'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_title(clickData):
    text = ''
    if clickData is not None:
        selected_country = clickData['points'][0]['location']
        text = str(selected_country)  + '-' + str(clickData['points'][0]['text'])
    return text

@app.callback(
    dash.dependencies.Output('horizontal-bar', 'figure'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_horizontal_bar(clickData):

    df_h = df_merged.groupby(by=['country','country_txt',
                                   'id', 'isOrganized']).agg({'eventid':'count'}).unstack(fill_value=0).stack().sort_values(by='country').reset_index()
    df_x = df_h.pivot_table(index=['country','country_txt'], columns='isOrganized')
    countries = df_h.drop_duplicates(subset=['country','country_txt']).reset_index()[['country','country_txt']]
    countries['organized'] = df_x['eventid']['organized'].reset_index()['organized']
    countries['unorganized'] = df_x['eventid']['unorganized'].reset_index()['unorganized']
    countries['Total'] = countries['organized'] +countries['unorganized']
    countries = countries.sort_values(by='Total', ascending= False)[0:15]


    fig = go.Figure()

    fig.add_trace(go.Bar(
        y=countries['country_txt'],
        x=countries['organized'],
        name='Organized',
        orientation='h',
        marker=dict(
            color='rgba(246, 78, 139, 0.6)',
            line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
        )
    ))
    fig.add_trace(go.Bar(
        y=countries['country_txt'],
        x=countries['unorganized'],
        name='Unorganized',
        orientation='h',
        marker=dict(
            color='rgba(58, 71, 80, 0.6)',
            line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
        )
    ))

    fig.update_layout(barmode='stack')
    
    return fig
        



@app.callback(
    [dash.dependencies.Output('incidents', 'figure'), dash.dependencies.Output('incident-box', 'style')],
    [dash.dependencies.Input('world-map', 'clickData')])
def update_incidents(clickData):
    df_agg = ''
    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        df_agg = df_merged[df_merged['id'] == selected_country].groupby(by=['iyear','country',
                                                                            'country_txt','id', 
                                                                            'isOrganized']).agg({
                                                                                        'eventid':'count'}).reset_index()
    fig = px.line(df_agg, x="iyear", y="eventid", color='isOrganized',
                  labels={
                     "iyear": "<b>Year</b>",
                     "eventid": "<b>Number of Incidents</b>",
                     "isOrganized": " "
                 }, title="<b>Number of Incidents by Years</b>")
    
    return fig,  {'visibility':'visible'}

@app.callback(
    [dash.dependencies.Output('fatalities', 'figure'), dash.dependencies.Output('fatalities-box', 'style')],
    [dash.dependencies.Input('world-map', 'clickData')])
def update_fatalities(clickData):
    df_agg = ''
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        df_agg = df_merged[df_merged['id'] == selected_country].groupby(by=['iyear','country',
                                                                        'country_txt','id']).agg({
                                                                                        'nkill':np.sum,
                                                                                        'nwound':np.sum  }).reset_index()
    # Add traces
    fig.add_trace(
        go.Line(x=df_agg['iyear'], y=df_agg['nwound'], name="Injuries", line=dict(color="#D4AC0D")),
        secondary_y=False,
    )

    fig.add_trace(
        go.Line(x=df_agg['iyear'], y=df_agg['nkill'], name="Fatalities", line=dict(color="#943126")),
        secondary_y=True,
    )     
    
    # Add figure title
    fig.update_layout(
        title_text="<b>Number of Injuries and Fatalities by Years</b>"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="<b>Year</b>")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Injuries</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Fatalities</b> ", secondary_y=True)

    
    return fig, {'visibility':'visible'}



    
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Dec/2021 22:03:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 22:03:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 22:03:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 22:03:06] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

127.0.0.1 - - [07/Dec/2021 22:03:06] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [07/Dec/2021 22:03:06] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Esra\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Esra\anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(fun

127.0.0.1 - - [07/Dec/2021 22:03:06] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [07/Dec/2021 22:06:54] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\Esra\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


C:\Users\Esra\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


127.0.0.1 - - [07/Dec/2021 22:06:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 22:06:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 22:06:55] "POST /_dash-update-component HTTP/1.1" 200 -
127